## Готовим данные

In [52]:
import pandas as pd

# Загрузка датасета
file_path = 'sentiment_texts.pickle'
data = pd.read_pickle(file_path)
data = data[['issuerid', 'SentimentScore', 'MessageText']]

# Показываем первые несколько строк датасета для оценки его структуры
data.head()

,issuerid,SentimentScore,MessageText
0,153,2,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...
1,230,4,Ozon продолжает развивать специализированные ф...
2,118,4,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...
3,220,5,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...
4,89,2,​​Windfall Tax — налог на сверхприбыль. Какие ...


In [53]:
import json
import pathlib
import nltk
from nltk import word_tokenize, SnowballStemmer
nltk.download('punkt')

num_companies = 276

file_path = '/content/dict.json'
with open(file_path, 'r', encoding='utf-8') as file:
    synon_dict = json.load(file)

stemmer = SnowballStemmer("russian")

markers = [f'%{i}%' for i in range(num_companies)]


def process_text(text, synon_dict):
    tokens = word_tokenize(text, language='russian')
    tokens = [token.lower() for token in tokens]

    stem_tokens = [stemmer.stem(token) for token in tokens if len(token) > 1]

    # Находим компании и заменяем на маркеры
    return replace_phrases_with_markers(stem_tokens, tokens, maximize_phrase_coverage(stem_tokens, synon_dict, 5), markers)


def maximize_phrase_coverage(tokens: list[str], dictionary, max_window_size=20) -> list[list[int]]:
    def binary_search(target: str) -> int:
        if target in dictionary:
            return len(target.split())  # Фраза найдена
        return 0  # Фраза не найдена

    # Фильтруем только непустые токены
    if len(tokens) == 0:
        return []
    token_ids, text_tokens = zip(*[[idx, token] for idx, token in enumerate(tokens) if token])
    token_count = len(text_tokens)

    # max_sums[token_idx] - максимальное значение суммы значений для префикса [0...token_idx]
    max_sums = [0] * token_count
    window_sizes = [-1] * token_count

    # Инициализация для первого токена
    max_sums[0] = binary_search(text_tokens[0])

    for token_idx in range(1, token_count):
        current_window = ''
        for sz in range(1, max_window_size + 1):
            prev_token_idx = token_idx - sz
            if prev_token_idx < -1:
                break

            if current_window == '':
                current_window = text_tokens[prev_token_idx + 1]
            else:
                current_window = text_tokens[prev_token_idx + 1] + ' ' + current_window

            coverage_value = binary_search(current_window)

            if current_window == 'moex' and text_tokens[prev_token_idx] == '(' and text_tokens[prev_token_idx + 2] == ':':
                continue

            if coverage_value != 0 and max_sums[token_idx] < max_sums[prev_token_idx] + coverage_value:
                max_sums[token_idx] = max_sums[prev_token_idx] + coverage_value
                window_sizes[token_idx] = sz

    # Восстановление размеров окон токенов
    bounds = []
    current_index = token_count - 1
    while current_index != -1:
        window_size = window_sizes[current_index]
        bounds.append(window_size)
        window_size = 1 if window_size == -1 else window_size
        current_index -= window_size
    bounds.reverse()

    # Восстановление групп токенов
    token_groups = []
    current_index = 0
    for window_size in bounds:
        if window_size == -1:
            current_index += 1
            continue
        group = token_ids[current_index:current_index + window_size]
        token_groups.append(group)
        current_index += window_size

    return token_groups


def replace_phrases_with_markers(stem_tokens, tokens, groups, markers):
    """
    Функция находит и заменяет группы токенов, соответствующие записям в словаре,
    на маркеры, а также сохраняет позиции маркеров в тексте.

    Параметры:
        tokens (list[str]): Исходный список слов.
        groups (list[list[int]]): Список групп индексов, соответствующих найденным фразам.
        markers (dict): Словарь маркеров, где ключ - это индекс фразы в dictionary, а значение - маркер.

    Возвращает:
        tuple: Модифицированный список слов и словарь с позициями маркеров.
    """
    new_tokens = []
    marker_positions = {}
    last_index = 0  # Следим за последним индексом, который был добавлен в new_tokens

    for group in groups:
        if not group:
            continue
        # Проверяем, соответствует ли группа записи в словаре
        phrase = " ".join(stem_tokens[idx] for idx in group)
        marker_index = synon_dict[phrase]  # Получаем маркер для найденной фразы, если он есть

        if marker_index and marker_index <= 274:
            # Добавляем все токены до начала текущей группы
            new_tokens.extend(stem_tokens[last_index:group[0]])
            # Добавляем маркер в новый список
            # new_tokens.append(markers[marker_index])
            # Записываем позицию маркера
            if marker_index not in marker_positions:
                marker_positions[marker_index] = []
            marker_positions[marker_index].append(len(new_tokens) - 1)

            # Обновляем последний обработанный индекс
            last_index = group[-1] + 1

    # Добавляем все оставшиеся токены после последней группы
    new_tokens.extend(stem_tokens[last_index:])

    return new_tokens, marker_positions


def extract_company_context(tokens, mentioned_companies, window_size=3):
    # Словарь для хранения контекстов для каждой компании
    company_contexts = {}

    # Перебираем все упоминания компаний
    for company_id, positions in mentioned_companies.items():
        # Список контекстов для текущей компании
        contexts = []

        for pos in positions:
            # Определяем начало и конец контекстного окна
            start = max(0, pos - window_size)
            end = min(len(tokens), pos + window_size + 1)

            # Собираем слова в окне вокруг вхождения компании
            context_tokens = tokens[start:end]

            # Добавляем полученный контекст в список контекстов компании
            for tok in context_tokens:
                contexts.append(tok)

        # Записываем список контекстов для текущей компании в общий словарь
        company_contexts[company_id] = contexts

    return company_contexts



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
tokens = []
labels = []
for index, row in data.iterrows():
    words, mentioned_companies = process_text(row.MessageText, synon_dict)
    company_context = extract_company_context(words, mentioned_companies, window_size=4)
    if (row.issuerid in company_context):
      tokens.append(company_context[row.issuerid])
      labels.append(row.SentimentScore)


len(labels)

8549

## Keras-модель

In [55]:
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers import Lambda
import keras.backend as K

max_features = 100000
maxlen = 100
batch_size = 64

In [56]:
# Инициализируем токенизатор
tokenizer = Tokenizer(num_words=max_features)

# Обучаем токенизатор на твитах
tokenizer.fit_on_texts(tokens)

# Преобразуем твиты в последовательности чисел
sequences = tokenizer.texts_to_sequences(tokens)

# Паддинг последовательностей до одинаковой длины
X = sequence.pad_sequences(sequences, maxlen=maxlen)

# Преобразуем labels в категориальный формат
y = to_categorical(labels)


# Убедитесь, что форма 'y' соответствует ожидаемой моделью
print("Форма y после to_categorical:", y.shape)

Форма y после to_categorical: (8549, 6)


In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
from keras import backend as K

model = Sequential()
model.add(Embedding(max_features, 256, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=15,
    verbose=1
)

result = model.predict(X)
result

Epoch 1/15
107/107 [==============================] - 14s 90ms/step - loss: 1.2811 - accuracy: 0.4547
Epoch 2/15
107/107 [==============================] - 8s 73ms/step - loss: 0.9188 - accuracy: 0.6562
Epoch 3/15
107/107 [==============================] - 5s 48ms/step - loss: 0.6314 - accuracy: 0.7902
Epoch 4/15
107/107 [==============================] - 4s 38ms/step - loss: 0.4607 - accuracy: 0.8519
Epoch 5/15
107/107 [==============================] - 3s 29ms/step - loss: 0.3554 - accuracy: 0.8865
Epoch 6/15
107/107 [==============================] - 2s 23ms/step - loss: 0.2851 - accuracy: 0.9139
Epoch 7/15
107/107 [==============================] - 2s 21ms/step - loss: 0.2334 - accuracy: 0.9308
Epoch 8/15
107/107 [==============================] - 2s 22ms/step - loss: 0.2017 - accuracy: 0.9376
Epoch 9/15
107/107 [==============================] - 2s 21ms/step - loss: 0.1750 - accuracy: 0.9465
Epoch 10/15
107/107 [==============================] - 2s 21ms/step - loss: 0.1576 - accur

array([[3.9039566e-03, 1.3717344e-04, 4.5761999e-01, 5.3700119e-01,
        1.0107091e-03, 3.2695112e-04],
       [3.1132934e-06, 2.4872580e-07, 1.6306485e-05, 1.5448061e-04,
        9.9981803e-01, 7.8490039e-06],
       [1.1795677e-05, 1.2820184e-06, 7.0119240e-05, 4.1928567e-04,
        9.9947649e-01, 2.1094249e-05],
       ...,
       [2.6214347e-04, 3.7921905e-05, 1.5435462e-04, 9.9804175e-01,
        1.4035498e-03, 1.0030330e-04],
       [5.1434090e-06, 5.4113349e-07, 2.1641061e-05, 4.0482698e-04,
        9.9955755e-01, 1.0200669e-05],
       [1.9550800e-05, 1.2787200e-04, 1.4323265e-04, 9.2468086e-05,
        3.6843278e-04, 9.9924845e-01]], dtype=float32)

In [59]:
import numpy as np

# Получение предсказаний модели
predictions = model.predict(X_test)

# Преобразование предсказаний в индексы классов
predicted_classes = np.argmax(predictions, axis=1)

# Преобразование истинных меток в индексы классов, если они в формате one-hot encoding
true_classes = np.argmax(y_test, axis=1)

# Расчет точности
accuracy_manual = np.mean(predicted_classes == true_classes)
print(f"Рассчитанная вручную точность: {accuracy_manual:.2f}")


54/54 [==============================] - 0s 4ms/step
Рассчитанная вручную точность: 0.61


In [60]:
import pickle
# Сохранение токенизатора в файл
with open('tokenizer_keras.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
model.save('my_model.h5')  # Создает HDF5 файл 'my_model.h5'

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## CatBoost-модель

In [62]:
!pip install catboost

In [63]:
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

# Пример данных
data = {
    'tokens': tokens,
    'labels': labels
}

df = pd.DataFrame(data)
df = df[df['labels'] != 0]

# Объединение токенов в строки
df['text'] = df['tokens'].apply(lambda x: ' '.join(x))

# Векторизация текста
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['labels']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Создание и обучение модели CatBoost
model = CatBoostClassifier(iterations=500, learning_rate=0.3, depth=6, loss_function='MultiClass')
model.fit(X_train, y_train)

# Оценка модели
print(model.score(X_test, y_test))

0:	learn: 1.4180806	total: 433ms	remaining: 3m 35s
1:	learn: 1.3151914	total: 787ms	remaining: 3m 15s
2:	learn: 1.2542001	total: 1.14s	remaining: 3m 9s
3:	learn: 1.2152037	total: 1.6s	remaining: 3m 18s
4:	learn: 1.1834288	total: 1.91s	remaining: 3m 9s
5:	learn: 1.1576126	total: 2.21s	remaining: 3m 2s
6:	learn: 1.1343285	total: 2.52s	remaining: 2m 57s
7:	learn: 1.1218620	total: 2.89s	remaining: 2m 57s
8:	learn: 1.1126719	total: 3.27s	remaining: 2m 58s
9:	learn: 1.1051159	total: 3.57s	remaining: 2m 54s
10:	learn: 1.0965229	total: 3.88s	remaining: 2m 52s
11:	learn: 1.0880916	total: 4.18s	remaining: 2m 49s
12:	learn: 1.0817894	total: 4.47s	remaining: 2m 47s
13:	learn: 1.0757290	total: 4.89s	remaining: 2m 49s
14:	learn: 1.0723778	total: 5.73s	remaining: 3m 5s
15:	learn: 1.0679367	total: 6.58s	remaining: 3m 19s
16:	learn: 1.0596051	total: 7.28s	remaining: 3m 26s
17:	learn: 1.0568557	total: 8.04s	remaining: 3m 35s
18:	learn: 1.0522985	total: 8.61s	remaining: 3m 37s
19:	learn: 1.0500924	total:

In [64]:
model.save_model('catboost_sentiment_model.cbm', format='cbm')

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump

# Настройка и обучение векторизатора здесь, если необходимо
dump(vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']

In [65]:
# Сохранение векторизатора
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

## Объединим 3 модели

In [74]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle
from catboost import CatBoostClassifier

# Загрузка токенизатора из файла
with open('tokenizer_keras.pickle', 'rb') as handle:
    tokenizer_keras = pickle.load(handle)

# Загрузка модели
    model_keras = load_model('my_model (1).h5')

def predict1(tokens):
    # Преобразование токенов в последовательности индексов
    sequences = tokenizer_keras.texts_to_sequences([tokens])

    # Паддинг последовательностей
    maxlen = 100  # Максимальная длина последовательности, как в модели
    padded_sequences = pad_sequences(sequences, maxlen=maxlen)

    # Сделать предсказание
    predictions = model_keras.predict(padded_sequences, verbose=0)
    predicted_class_index = np.argmax(predictions)
    predicted_class = predicted_class_index + 1
    return predicted_class



# Загрузка TF-IDF векторизатора
with open('tfidf_vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

# Загрузка модели CatBoost
model_catboost = CatBoostClassifier()
model_catboost.load_model('catboost_sentiment_model.cbm')


def predict2(tokens):
    text = ' '.join(tokens)
    transformed = loaded_vectorizer.transform([text])
    predictions = model_catboost.predict(transformed)

    return predictions[0]


In [67]:
def predict(tokens):
    return round(0.51 * predict2(tokens)) # + 0.49 * predict2(tokens))

In [75]:
for i in range(1000):
  predict1(['asd', 'повыс', 'приб', 'очен'])